# 4교시 2. CNN과 전이학습

## 1. 소규모 데이터셋으로 만드는 강력한 학습 모델
- 데이터셋이 많지 않을 때

### 치매 환자의 뇌인지 일반인의 뇌인지 예측하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers

import numpy as np
import matplotlib.pyplot as plt

# 깃허브에 준비된 데이터를 가져옵니다.
!git clone https://github.com/taehojo/data-ch20.git

# 학습셋의 변형을 설정하는 부분입니다. -> 또 다른 특징을 찾아낼 수 있다. (정규화)
train_datagen = ImageDataGenerator(rescale=1./255,          # 픽셀의 값을 0.0~1.0사이로 정규화합니다.
                                  horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,    # 전체 크기의 10% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1)   # 마찬가지로 위, 아래로 이동합니다.

train_generator = train_datagen.flow_from_directory(
       './data-ch20/train',   # 학습셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# 테스트셋은 이미지 부풀리기 과정을 진행하지 않습니다.
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
       './data-ch20/test',   # 테스트셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# 앞서 배운 CNN 모델을 만들어 적용해 보겠습니다. h=150, w=150, in_channel=3 (colors)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# 모델의 실행 옵션을 설정합니다.
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델 최적화를 위한 설정 구간입니다.
modelpath="./data-ch20/MNIST_CNN.keras"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델을 실행합니다.
history = model.fit(train_generator, ##
                    validation_data=test_generator, ##
                    epochs=30,
                    verbose=1,
                    callbacks=[early_stopping_callback, checkpointer])

# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(test_generator)[1]))


fatal: destination path 'data-ch20' already exists and is not an empty directory.
Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 72, 72, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │     1,183,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,212,513 (4.63 MB)

 Trainable params: 1,212,513 (4.63 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5008 - loss: 0.7474
Epoch 1: val_loss improved from inf to 0.69399, saving model to ./data-ch20/MNIST_CNN.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.4996 - loss: 0.7454 - val_accuracy: 0.5000 - val_loss: 0.6940
Epoch 2/30
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5297 - loss: 0.6873
Epoch 2: val_loss improved from 0.69399 to 0.69231, saving model to ./data-ch20/MNIST_CNN.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.5283 - loss: 0.6875 - val_accuracy: 0.5000 - val_loss: 0.6923
Epoch 3/30
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5500 - loss: 0.6907
Epoch 3: val_loss improved from 0.69231 to 0.69157, saving model to ./data-ch20/MNIST_CNN.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.5477 - loss: 0.6909 - val_accuracy: 0.5000 - val_loss: 0.6916
Epoch 4/30
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5393 - loss: 0.6925
Epoch 4: val_loss improved f

## 2. 전이 학습으로 모델 성능 극대화하기

- 기존에 큰 데이터로 잘 학습한 결과를 이용하여, 나의 데이터에 맞는 미세조정을 함으로써 내가 가진 데이터만으로 좋은 결과를 내는 학습 방식
- 랜덤보다 좋음
- 왜? **면/선 등을 이미 image net을 통해 파악**
- image net에 뇌 mri가 없어도 초깃값으로 세팅된 모델보다 더 성능이 좋게 나온다.



### 실습: 전이 학습 실습하기

<img src="https://github.com/taehojo/fastcampus_ai/blob/master/data/img/04-04.png?raw=1" width="400"/>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

# 학습셋의 변형을 설정하는 부분입니다.
train_datagen = ImageDataGenerator(rescale=1./255,          # 주어진 이미지의 크기를 설정합니다.
                                  horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,    # 전체 크기의 10% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1)   # 마찬가지로 위, 아래로 이동합니다.

train_generator = train_datagen.flow_from_directory(
       './data-ch20/train',
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# 테스트셋의 정규화를 설정합니다.
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
       './data-ch20/test',
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# VGG16 모델을 불러옵니다.
## include_top=False (vgg16의 image classification을 위해 달았던 FC 포함하지 않음)
## 그리고 pre-trained model은 train하지 않음 -> transfermodel.trainable=False
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
transfer_model.trainable = False

# 우리의 모델을 설정합니다.
finetune_model = models.Sequential()
finetune_model.add(transfer_model) #가지고오기
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dropout(0.5))
finetune_model.add(Dense(1, activation='sigmoid'))

# 모델의 실행 옵션을 설정합니다.
finetune_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 학습의 조기 중단을 설정합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

# 모델을 실행합니다.
history = finetune_model.fit(
       train_generator,
       epochs=30,
       validation_data=test_generator,
       validation_steps=10,
       callbacks=[early_stopping_callback])

# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (finetune_model.evaluate(test_generator)[1]))


Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.6075 - loss: 1.1265 - val_accuracy: 0.7400 - val_loss: 0.6172
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.6044 - loss: 0.6429 - val_accuracy: 0.6000 - val_loss: 0.5931
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 0.6646 - loss: 0.6122 - val_accuracy: 0.8400 - val_loss: 0.5333
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.8536 - loss: 0.5054 - val_accuracy: 0.8800 - val_loss: 0.4118
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.7225 - loss: 0.5134 - val_accuracy: 0.9000 - val_loss: 0.4788
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.7944 - loss: 0.4859 - val_accuracy: 0.8600 - val_loss: 0.4329
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.7834 - loss: 0.4412 - val_accuracy: 0.94